In [1]:
#!pip install -q mlxtend

import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
# (opciono) fokus na parove i trojke
#freq = freq[(freq["itemset_len"] >= 2) & (freq["itemset_len"] <= 3)]


In [2]:
# 1) Učitaj podatke
df = pd.read_csv("Reviews_sample_balanced.csv")

In [3]:
# 2) (Preporuka) ukloni duplikate istog (UserId, ProductId)
df = df.drop_duplicates(subset=["UserId", "ProductId"])

In [4]:
# 3) Napravi transakcije: lista proizvoda po korisniku
transactions = (
    df.groupby("UserId")["ProductId"]
      .apply(lambda s: sorted(set(s)))
      .tolist()
)
print("Broj transakcija (korisnika):", len(transactions))


Broj transakcija (korisnika): 9252


In [5]:
# 4) One-hot encoding za Apriori
te = TransactionEncoder()
X = te.fit(transactions).transform(transactions)
basket = pd.DataFrame(X, columns=te.columns_)

In [6]:
# 5) Frequent itemsets (min_support podesi!)
# Za ovako veliki skup, kreni vrlo nisko (npr. 0.002 = 0.2% korisnika)
freq = apriori(basket, min_support=0.002, use_colnames=True)
freq["itemset_len"] = freq["itemsets"].apply(len)

In [7]:
# 6) Association rules
rules = association_rules(freq, metric="lift", min_threshold=1.2)

In [8]:
# 7) Sortiranje i prikaz top pravila
rules = rules.sort_values(["lift", "confidence"], ascending=False)


In [9]:
def fs(s):  # frozenset -> string
    return ", ".join(sorted(list(s)))

out = rules[["antecedents", "consequents", "support", "confidence", "lift"]].copy()
out["antecedents"] = out["antecedents"].apply(fs)
out["consequents"] = out["consequents"].apply(fs)

print(out.head(20).to_string(index=False))

Empty DataFrame
Columns: [antecedents, consequents, support, confidence, lift]
Index: []


In [15]:
lens = pd.Series([len(t) for t in transactions])
print(lens.describe())
print("Korisnika sa 1 proizvodom:", (lens==1).sum())
print("Korisnika sa >=2 proizvoda:", (lens>=2).sum())
print("Korisnika sa >=3 proizvoda:", (lens>=3).sum())
print("Korisnika sa >=4 proizvoda:", (lens>=4).sum())


count    9252.000000
mean        1.080199
std         0.356312
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         8.000000
dtype: float64
Korisnika sa 1 proizvodom: 8668
Korisnika sa >=2 proizvoda: 584
Korisnika sa >=3 proizvoda: 105
Korisnika sa >=4 proizvoda: 32


In [13]:
# !pip install -q mlxtend

import time
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

t0 = time.time()
def log(msg):
    print(f"[{time.time()-t0:7.1f}s] {msg}", flush=True)

# =========================
# 1) Ucitaj podatke
# =========================
log("Ucitavam podatke...")
df = pd.read_csv("Reviews_sample_balanced.csv")
log(f"Ucitano redova: {len(df):,}")

# Provera da postoje kolone
needed = {"UserId", "ProductId"}
missing = needed - set(df.columns)
if missing:
    raise ValueError(f"Nedostaju kolone: {missing}. Kolone koje imam: {list(df.columns)}")

# =========================
# 2) Ukloni duplikate (isti korisnik isti proizvod vise puta)
# =========================
log("Uklanjam duplikate (UserId, ProductId)...")
df = df.drop_duplicates(subset=["UserId", "ProductId"])
log(f"Nakon duplikata redova: {len(df):,}")

# =========================
# 3) Napravi transakcije: lista proizvoda po korisniku
# =========================
log("Pravim transakcije (ProductId po UserId)...")
transactions = (
    df.groupby("UserId")["ProductId"]
      .apply(lambda s: list(pd.unique(s)))
      .tolist()
)
log(f"Ukupno transakcija (korisnika): {len(transactions):,}")

# Provera duzina transakcija
lens = pd.Series([len(t) for t in transactions])
log("Statistika duzine transakcija:")
print(lens.describe())
print("Korisnika sa 1 proizvodom:", int((lens == 1).sum()))
print("Korisnika sa >=2 proizvoda:", int((lens >= 2).sum()))

# =========================
# 4) FILTRIRAJ: zadrzi samo korisnike sa >=2 proizvoda
# =========================
log("Filtriram transakcije (zadrzavam >=2 proizvoda)...")
transactions2 = [t for t in transactions if len(t) >= 2]
log(f"Transakcije posle filtriranja: {len(transactions2):,}")

if len(transactions2) < 50:
    log("Upozorenje: jako malo transakcija sa >=2 proizvoda -> moguce malo ili nimalo pravila.")

# =========================
# 5) One-hot encoding za Apriori
# =========================
log("Radim TransactionEncoder (one-hot)...")
te = TransactionEncoder()
X = te.fit(transactions2).transform(transactions2)
basket = pd.DataFrame(X, columns=te.columns_)
log(f"One-hot shape: {basket.shape[0]} x {basket.shape[1]} (transakcije x proizvodi)")

# =========================
# 6) Apriori: frequent itemsets
# =========================
# Support sada racunas u odnosu na transactions2 (npr. 584), pa mozes vece vrednosti
min_sup = 0.005    # 1% od 584 ~ 6 transakcija
max_len = 2        # fokus na parove (brze i smisleno za seminarski)

log(f"Krecem Apriori (min_support={min_sup}, max_len={max_len})...")
freq = apriori(
    basket,
    min_support=min_sup,
    use_colnames=True,
    low_memory=True,
    max_len=max_len
)
freq["itemset_len"] = freq["itemsets"].apply(len)

log(f"Frequent itemsets ukupno: {len(freq):,}")
log(f"Frequent itemsets len=2: {(freq['itemset_len']==2).sum():,}")

# Ako nema parova, spusti support
if (freq["itemset_len"] == 2).sum() == 0:
    log("Nema parova sa datim supportom. Probaj min_support=0.005 ili 0.002.")
    # Ne prekidam program, samo ispisujem.

# =========================
# 7) Association rules
# =========================
log("Racunam association rules...")
rules = association_rules(freq, metric="lift", min_threshold=1.1)
log(f"Pravila ukupno: {len(rules):,}")

# =========================
# 8) Prikaz top pravila
# =========================
def fs(s):
    return ", ".join(sorted(list(s)))

if len(rules) == 0:
    print("\nNEMA PRAVILA sa ovim pragovima.")
    print("Probaj:")
    print("- spusti min_support na 0.005 ili 0.002")
    print("- smanji min_threshold (lift) na 1.05")
else:
    rules = rules.sort_values(["lift", "confidence"], ascending=False)

    out = rules[["antecedents", "consequents", "support", "confidence", "lift"]].copy()
    out["antecedents"] = out["antecedents"].apply(fs)
    out["consequents"] = out["consequents"].apply(fs)

    print("\nTOP 20 pravila:")
    print(out.head(20).to_string(index=False))


[    0.0s] Ucitavam podatke...
[    0.1s] Ucitano redova: 10,000
[    0.1s] Uklanjam duplikate (UserId, ProductId)...
[    0.1s] Nakon duplikata redova: 9,994
[    0.1s] Pravim transakcije (ProductId po UserId)...
[    0.5s] Ukupno transakcija (korisnika): 9,252
[    0.5s] Statistika duzine transakcija:
count    9252.000000
mean        1.080199
std         0.356312
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         8.000000
dtype: float64
Korisnika sa 1 proizvodom: 8668
Korisnika sa >=2 proizvoda: 584
[    0.5s] Filtriram transakcije (zadrzavam >=2 proizvoda)...
[    0.5s] Transakcije posle filtriranja: 584
[    0.5s] Radim TransactionEncoder (one-hot)...
[    0.5s] One-hot shape: 584 x 1038 (transakcije x proizvodi)
[    0.5s] Krecem Apriori (min_support=0.005, max_len=2)...
[    0.6s] Frequent itemsets ukupno: 71
[    0.6s] Frequent itemsets len=2: 2
[    0.6s] Racunam association rules...
[    0.6s] Pravila ukupno: 4

TOP 20 pravila:
ante

In [6]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import pandas as pd

def run_apriori(transactions, min_items, min_sup, min_lift=1.1):
    print(f"\n=== Korisnici sa >={min_items} proizvoda ===")

    trans = [t for t in transactions if len(t) >= min_items]
    print("Broj transakcija:", len(trans))

    if len(trans) < 10:
        print("Previse malo transakcija, preskacem.")
        return None

    te = TransactionEncoder()
    X = te.fit(trans).transform(trans)
    basket = pd.DataFrame(X, columns=te.columns_)

    freq = apriori(
        basket,
        min_support=min_sup,
        use_colnames=True,
        max_len=2,
        low_memory=True
    )

    freq["itemset_len"] = freq["itemsets"].apply(len)
    print("Parovi (len=2):", (freq["itemset_len"] == 2).sum())

    if (freq["itemset_len"] == 2).sum() == 0:
        print("Nema parova sa datim supportom.")
        return None

    rules = association_rules(freq, metric="lift", min_threshold=min_lift)
    rules = rules.sort_values(["lift", "confidence"], ascending=False)

    def fs(s):
        return ", ".join(sorted(list(s)))

    out = rules[["antecedents", "consequents", "support", "confidence", "lift"]].copy()
    out["antecedents"] = out["antecedents"].apply(fs)
    out["consequents"] = out["consequents"].apply(fs)

    print(out.head(10).to_string(index=False))
    return out


In [7]:
# >=2 (vec imas, ali moze i ovde)
rules_2 = run_apriori(transactions, min_items=2, min_sup=0.002)

# >=3 proizvoda (105 korisnika)
rules_3 = run_apriori(transactions, min_items=3, min_sup=0.02)
# 0.02 * 105 ≈ 2 korisnika

# >=4 proizvoda (32 korisnika)
rules_4 = run_apriori(transactions, min_items=4, min_sup=0.06)
# 0.06 * 32 ≈ 2 korisnika



=== Korisnici sa >=2 proizvoda ===
Broj transakcija: 584
Parovi (len=2): 31
antecedents consequents  support  confidence  lift
 B003NCEB2K  B00017LEXO 0.003425         1.0 292.0
 B00017LEXO  B003NCEB2K 0.003425         1.0 292.0
 B0012NUVN0  B0009YJ4CW 0.003425         1.0 292.0
 B0009YJ4CW  B0012NUVN0 0.003425         1.0 292.0
 B000VK339Y  B001PMC3PM 0.003425         1.0 292.0
 B001PMC3PM  B000VK339Y 0.003425         1.0 292.0
 B001BORBHO  B0051ZCNIQ 0.003425         1.0 292.0
 B0051ZCNIQ  B001BORBHO 0.003425         1.0 292.0
 B001LQCOIS  B004WTHCO2 0.003425         1.0 292.0
 B004WTHCO2  B001LQCOIS 0.003425         1.0 292.0

=== Korisnici sa >=3 proizvoda ===
Broj transakcija: 105
Parovi (len=2): 0
Nema parova sa datim supportom.

=== Korisnici sa >=4 proizvoda ===
Broj transakcija: 32
Parovi (len=2): 0
Nema parova sa datim supportom.
